<a href="https://colab.research.google.com/github/b2204yo/supervisedPetClass/blob/develop/supervisedPetClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

Downloading the data

In [27]:
from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c dogs-vs-cats

!unzip -qq dogs-vs-cats.zip

!unzip -qq train.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace train/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


Copying images to training, validation, and test directories

In [29]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

FileExistsError: ignored

Select learning model

Set picuter setting

In [49]:
learning_model_name = "VGG16"
# VGG16 : 180, Efficient Net 224

if learning_model_name == "VGG16" or "ResNet":
  num_picel_units = 180
  num_epoch = 30
  conv_base  = keras.applications.vgg16.VGG16(
     weights="imagenet",
     include_top=False)

elif learning_model_name == "EfficientNetV2":
  num_picel_units = 224
  num_epoch = 350
  conv_base  = keras.applications.vgg16.VGG16(
     weights="imagenet",
     include_top=False)

Data preprocessing

In [50]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(num_picel_units, num_picel_units),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(num_picel_units, num_picel_units),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(num_picel_units, num_picel_units),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [51]:
import numpy as np
import tensorflow as tf
random_numbers = np.random.normal(size=(1000, 16))
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)

In [52]:
for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break

(16,)
(16,)
(16,)


In [53]:
batched_dataset = dataset.batch(32)
for i, element in enumerate(batched_dataset):
    print(element.shape)
    if i >= 2:
        break

(32, 16)
(32, 16)
(32, 16)


In [54]:
reshaped_dataset = dataset.map(lambda x: tf.reshape(x, (4, 4)))
for i, element in enumerate(reshaped_dataset):
    print(element.shape)
    if i >= 2:
        break

(4, 4)
(4, 4)
(4, 4)


Displaying the shapes of the data and labels yielded by the Dataset

In [55]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

data batch shape: (32, 180, 180, 3)
labels batch shape: (32,)


Using image_dataset_from_directory to read images

In [56]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(num_picel_units, num_picel_units),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(num_picel_units, num_picel_units),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(num_picel_units, num_picel_units),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


Instantiating the VGG16,ResNet, Mobilenet, Efficient Net convolutional base

In [ ]:
# VGG16
# if learning_model_name == "VGG16":
#   conv_base  = keras.applications.vgg16.VGG16(
#      weights="imagenet",
#      include_top=False)

# # Efficient
# elif learning_model_name == "EfficientNetV2":
#   conv_base = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(weights="imagenet")
conv_base.summary()
conv_base.trainable = False

Printing the list of trainable weights before and after freezing

In [38]:
conv_base.trainable = True
print("This is the number of trainable weights "
      "before freezing the conv base:", len(conv_base.trainable_weights))

This is the number of trainable weights before freezing the conv base: 243


In [39]:
conv_base.trainable = False
print("This is the number of trainable weights "
      "after freezing the conv base:", len(conv_base.trainable_weights))

This is the number of trainable weights after freezing the conv base: 0


Adding a data augmentation stage and a classifier to the convolutional base

In [40]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(num_picel_units, num_picel_units, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

Evaluating the model on the test set

In [ ]:
conv_base.summary()

Freezing all layers until the fourth from the last

In [42]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

Fine-tuning the model

In [46]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=num_epoch,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/50
63/63 [==============================] - 18s 141ms/step - loss: 0.6800 - accuracy: 0.6990 - val_loss: 0.6672 - val_accuracy: 0.8430
Epoch 2/50
63/63 [==============================] - 6s 96ms/step - loss: 0.6794 - accuracy: 0.7170 - val_loss: 0.6659 - val_accuracy: 0.8420
Epoch 3/50
63/63 [==============================] - 6s 97ms/step - loss: 0.6786 - accuracy: 0.7025 - val_loss: 0.6646 - val_accuracy: 0.8440
Epoch 4/50
63/63 [==============================] - 10s 149ms/step - loss: 0.6779 - accuracy: 0.7140 - val_loss: 0.6632 - val_accuracy: 0.8510
Epoch 5/50
63/63 [==============================] - 6s 98ms/step - loss: 0.6774 - accuracy: 0.7170 - val_loss: 0.6617 - val_accuracy: 0.8490
Epoch 6/50
63/63 [==============================] - 7s 108ms/step - loss: 0.6758 - accuracy: 0.7225 - val_loss: 0.6602 - val_accuracy: 0.8500
Epoch 7/50
63/63 [==============================] - 7s 109ms/step - loss: 0.6757 - accuracy: 0.7320 - val_loss: 0.6586 - val_accuracy: 0.8500
Epoch 8

In [ ]:
model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

Plot result graph

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()